In [80]:
import numpy as np
import hdf5storage

from Neural_Decoding.preprocessing_funcs import bin_spikes
from Neural_Decoding.preprocessing_funcs import bin_output
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import tensorflow.keras
from keras.models import Sequential
from keras.layers import LSTM
#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.decoders import KalmanFilterDecoder
from scipy.optimize import curve_fit
%matplotlib qt

In [2]:
m = {}
mat = hdf5storage.loadmat('indy_20170131_02.mat')
for k, v in mat.items():
    m[k] = np.transpose(v)

In [33]:
tmp = m['spikes']
time = m['t'].squeeze()
loca = m['finger_pos']
cursor = m['cursor_pos']
target = m['target_pos']

In [6]:
print(tmp[0][0].shape)
print(tmp[1][4].shape)
print(tmp[2][4].shape)
print(tmp[3][4].shape)
print(tmp[4][4].shape)

(1973, 1)
(879, 1)
(0, 0)
(0, 0)
(0, 0)


In [4]:
spikes = []

In [5]:
for j in range(tmp.shape[1]):
    for i in range(tmp.shape[0]):
        if (tmp[i][j].shape[0] != 0):
            spikes.append(tmp[i][j].squeeze())

In [6]:
def plot_raster(spikes):
    n_neurons = len(spikes)
    ax = plt.gca()
    colors = 'r'

    kwargs = {}
    kwargs.setdefault("linestyle", "None")
    kwargs.setdefault("marker", "|")
    # Default markersize determined by matching eventplot
    fig = ax.figure
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    ax_height = bbox.height * fig.dpi
    markersize = max(ax_height * 0.8 / n_neurons, 1)
    # For 1 - 3 neurons, we need an extra fudge factor to match eventplot
    markersize -= max(4 - n_neurons, 0) ** 2 * ax_height * 0.005
    kwargs.setdefault("markersize", markersize)
    kwargs.setdefault("markeredgewidth", 1)

    for i in range(n_neurons):
        spiketimes = spikes[i].ravel()
        ax.plot(
            spiketimes,
            np.zeros_like(spiketimes) + (i + 1),
            color=colors,
            **kwargs,
        )

    # --- set axes limits
    ax.set_xlim(1650, 2500)
    ax.set_ylim(0, n_neurons)
    ax.set_xlabel('time')
    ax.set_title('raster')
    ax.set_ylabel('unit')
    plt.xlim(1650, 2500)

In [22]:
def plot_psth(bins, dt, t_start, t_end):
    # Plot the PSTH
    x = np.arange(t_start, t_end - dt, dt)
    fig, ax = plt.subplots()
    ax.bar(x, bins, align='edge')
    ax.set_xlabel('Time (sec)')
    ax.set_ylabel('Number of spikes/bin')
    ax.set_title('Peri-Stimulus Time Histogram')
    plt.show()

In [8]:
def dif(arr, axis, period):
    nd = arr.ndim
    slice1 = [slice(None)] * nd
    slice2 = [slice(None)] * nd
    slice1[axis] = slice(period, None)
    slice2[axis] = slice(None, 0 - period)
    slice1 = tuple(slice1)
    slice2 = tuple(slice2)
    return np.subtract(arr[slice1], arr[slice2])

In [26]:
loca_diff = dif(loca, 1, 2)
time_diff = dif(time, 0, 2)
curs_diff = dif(cursor, 1, 2)

In [27]:
vel = np.divide(loca_diff, time_diff)
curs_vel = np.divide(curs_diff, time_diff)

In [28]:
vel=np.concatenate((vel[:, 0 : 1], vel, vel[:, -1 :]),axis=1)
curs_vel=np.concatenate((curs_vel[:, 0 : 1], curs_vel, curs_vel[:, -1 :]),axis=1)

In [29]:
vel = vel.T
curs_vel = curs_vel.T

In [197]:
dt=0.1 #Size of time bins (in seconds)
t_start=time[0] #Time to start extracting data - here the first time velocity was recorded
t_end=time[-1] #Time to finish extracting data - here the last time velocity was recorded
downsample_factor=1 #Downsampling of output (to make binning go faster). 1 means no downsampling.

In [198]:
spikes = np.array(spikes, dtype = object)

In [199]:
for i in range(spikes.shape[0]):
    spikes[i]=np.squeeze(spikes[i])

In [200]:
###Preprocessing to put spikes and output in bins###

#Bin neural data using "bin_spikes" function
neural_data=bin_spikes(spikes,dt,t_start,t_end)

#Bin output (velocity/position) data using "bin_output" function
vels_binned=bin_output(vel,time,dt,t_start,t_end,downsample_factor)
curs_vel_binned=bin_output(curs_vel,time,dt,t_start,t_end,downsample_factor)
pos_binned=bin_output(loca.T,time,dt,t_start,t_end,downsample_factor)
# target_binned=bin_output(target.T,time,dt,t_start,t_end,downsample_factor)
# curs_binned=bin_output(cursor.T,time,dt,t_start,t_end,downsample_factor)

#We will now determine acceleration    
temp=np.diff(vels_binned,axis=0) #The acceleration is the difference in velocities across time bins 
acc_binned=np.concatenate((temp,temp[-1:,:]),axis=0) #Assume acceleration at last time point is same as 2nd to last
#We will now determine acceleration for cursor
temp=np.diff(curs_vel_binned,axis=0) #The acceleration is the difference in velocities across time bins 
curs_acc_binned=np.concatenate((temp,temp[-1:,:]),axis=0) #Assume acceleration at last time point is same as 2nd to last

In [185]:
#caculate all spikes
sum_nur = np.sum(neural_data, axis=1)
plot_psth(sum_nur, dt, t_start, t_end)

In [213]:
ve_num = np.sqrt(np.add(np.multiply(curs_vel_binned[:, 0], curs_vel_binned[:, 0]), np.multiply(curs_vel_binned[:, 1], curs_vel_binned[:, 1])))
acc_num = np.sqrt(np.add(np.multiply(curs_acc_binned[:, 0], curs_acc_binned[:, 0]), np.multiply(curs_acc_binned[:, 1], curs_acc_binned[:, 1])))

In [204]:
def func(x, a, b, c):
  return a * np.exp(-b * x) + c

In [215]:
#popt, pcov = curve_fit(func, ve_num, neural_data[:, 2])  #for vec
#nur_pred = [func(i, popt[0], popt[1], popt[2]) for i in ve_num]
popt, pcov = curve_fit(func, acc_num, neural_data[:, 2])  #for acc
nur_pred = [func(i, popt[0], popt[1], popt[2]) for i in acc_num]

In [217]:
# 数据点与原先的进行画图比较
ax = plt.subplot()
plt.plot(acc_num, neural_data[:, 2], '*', label='original values')
plt.plot(acc_num, nur_pred, 'r', label='fit values')
ax.set_xlabel('Acc')
ax.set_ylabel('Number of spikes')
ax.set_title('Tunning curve')
plt.draw()

In [218]:
#task2 find the units with low r2_score
del_ind = []
sc = []
std_vels = StandardScaler().fit_transform(vels_binned)
std_neu = StandardScaler().fit_transform(neural_data)
for i in range(neural_data.shape[1]):
    reg = LinearRegression().fit(std_vels, std_neu[:,i])
    r2 = reg.score(std_vels, std_neu[:,i])
    if ( r2 < 0.1):
        del_ind.append(i)
        sc.append(r2)

In [230]:
ax = plt.subplot()
plt.plot(del_ind, sc, 'r')
ax.set_xlabel('unit id')
ax.set_ylabel('R2')
ax.set_title('R2 scores')
plt.draw()

In [219]:
sco = np.array(sc)
p7 = np.percentile(sco, 35)
del_ind = np.array(del_ind)

In [220]:
neural_data = np.delete(neural_data, del_ind[sco < p7], 1)

In [221]:
print(loca.shape)
neural_data.shape

(3, 203995)


(8159, 174)

In [257]:
#task 3
#The covariate is simply the matrix of firing rates for all neurons over time
X=neural_data
#The final output covariates include position, velocity, and acceleration
#y=np.concatenate((pos_binned,vels_binned,acc_binned),axis=1)  #pos、vec、acc
#y=np.concatenate((pos_binned,vels_binned),axis=1)  #pos、vec
#y=pos_binned  #pos
#y=vels_binned  #pos  #vec
y=acc_binned  #pos  #acc

In [258]:
def process_data(X, y):
    #Set what part of data should be part of the training/testing/validation sets
    training_range=[0, 0.7]
    testing_range=[0.7, 0.85]
    valid_range=[0.85,1]
    #Number of examples after taking into account bins removed for lag alignment
    num_examples=X.shape[0]

    #Note that each range has a buffer of 1 bin at the beginning and end
    #This makes it so that the different sets don't include overlapping data
    training_set=np.arange(np.int_(np.round(training_range[0]*num_examples))+1,np.int_(np.round(training_range[1]*num_examples))-1)
    testing_set=np.arange(np.int_(np.round(testing_range[0]*num_examples))+1,np.int_(np.round(testing_range[1]*num_examples))-1)
    valid_set=np.arange(np.int_(np.round(valid_range[0]*num_examples))+1,np.int_(np.round(valid_range[1]*num_examples))-1)

    #Get training data
    X_train=X[training_set,:]
    y_train=y[training_set,:]

    #Get testing data
    X_test=X[testing_set,:]
    y_test=y[testing_set,:]

    #Get validation data
    X_valid=X[valid_set,:]
    y_valid=y[valid_set,:]
    #Z-score inputs 
    X_train_mean=np.nanmean(X_train,axis=0)
    X_train_std=np.nanstd(X_train,axis=0)
    X_train=(X_train-X_train_mean)/X_train_std
    X_test=(X_test-X_train_mean)/X_train_std
    X_valid=(X_valid-X_train_mean)/X_train_std

    #Zero-center outputs
    y_train_mean=np.mean(y_train,axis=0)
    y_train=y_train-y_train_mean
    y_test=y_test-y_train_mean
    y_valid=y_valid-y_train_mean
    return [X_train, X_test, X_valid, y_train, y_test, y_valid, y_train_mean]

In [259]:
X_train, X_test, X_valid, y_train, y_test, y_valid, y_train_mean = process_data(X, y)

In [260]:
#Declare model
model=KalmanFilterDecoder(C=3) #There is one optional parameter that is set to the default in this example (see ReadMe)

#Fit model
model.fit(X_train,y_train)

#Get predictions
y_valid_predicted=model.predict(X_valid,y_valid)

#Get metrics of fit (see read me for more details on the differences between metrics)
#First I'll get the R^2
R2=r2_score(y_valid,y_valid_predicted)
print('R2:',R2) #I'm just printing the R^2's of the 3rd and 4th entries that correspond to the velocities

R2: 0.15661480483391566


In [33]:
#As an example, I plot an example 1000 values of the x velocity (column index 2), both true and predicted with the Kalman filter
#Note that I add back in the mean value, so that both true and predicted values are in the original coordinates
fig_x=plt.figure()
plt.plot(y_valid[1000:2000,2]+y_train_mean[2],'b')
plt.plot(y_valid_predicted[1000:2000,2]+y_train_mean[2],'r')
#Save figure
# fig_x.savefig('x_velocity_decoding.eps')

In [231]:
bins_before=0 #How many bins of neural data prior to the output are used for decoding
bins_current=1 #Whether to use concurrent time bin of neural data
bins_after=0 #How many bins of neural data after the output are used for decoding
# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X=get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)
y=vels_binned
X_train, X_test, X_valid, y_train, y_test, y_valid, y_train_mean = process_data(X, y)

In [232]:
#Declare model
model_lstm=LSTMDecoder(units=400,dropout=0,num_epochs=5)

#Fit model
model_lstm.fit(X_train,y_train)

#Get predictions
y_valid_predicted_lstm=model_lstm.predict(X_valid)

#Get metric of fit
R2_lstm=r2_score(y_valid,y_valid_predicted_lstm)
print('R2:', R2_lstm)

39/39 [==============================] - 0s 2ms/step
R2: 0.4266074244179127


In [233]:
X = neural_data
y = vels_binned
X_train, X_test, X_valid, y_train, y_test, y_valid, y_train_mean = process_data(X, y)
line_mod = LinearRegression().fit(X_train, y_train)
y_valid_predicted_line = line_mod.predict(X_valid)
R2_line = r2_score(y_valid, y_valid_predicted_line)
print('R2:', R2_line)

R2: 0.3277089127111392
